In [1]:
import ee

import apgis
from apgis import Date
from apgis.apresource import Resource as Resource

import geemap
Map = geemap.Map(zoom=1)
Map.add_basemap("SATELLITE")

ModuleNotFoundError: No module named 'apgis'

## Initialize EE and Display list of remote Resources

In [2]:
apgis.eeInitialize()
Resource.showAvailable()

1.karayambedu1_L.geojson
2.karayambedu2_S.geojson
3.karayambedu3_S.geojson
4.karayambedu4_P.geojson
5.lanka_XXXL.geojson
6.mambakkam1_L.geojson
7.mettupalayam1_S.geojson
8.mettupalayam2_S.geojson
9.mettupalayam3_S.geojson
10.mettupalayam4_M.geojson
11.mettupalayam5_P.geojson
12.valparai1_M.geojson
13.valparai2_P.geojson


## Generate Collection and Base Image

In [40]:
remotefileName = "mettupalayam4_M"

drange = (Date("2020-01-10"), Date("2020-01-30"))
resource = Resource(remotefile=f"{remotefileName}.geojson")

S2 = apgis.gee.genCollection(sensor="L2A", daterange=drange, geometry=resource.field.eeROI)
print(S2.size().getInfo())

image = S2.first().clip(resource.field.eeROI.buffer(2000))

4


## Visual Parameters


In [41]:
S2TC = {
    'min': 0, 
    'max': 255,
    'bands': ['TCI_R', 'TCI_G', 'TCI_B']
}

ndmiVis = {
   'min': -1, 
   'max': 1, 
   'palette': ['FFFFFF', '513927', '5B5430', '587147', '438F72', '00AAAD', '006666', '004C4C', '000000']
}

ndviVis = {
   'min': 0.0, 
   'max': 1.0, 
   'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000', '529400', 
               '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301']
}

ndvi7Vis = {
    'min': 0.0, 
    'max': 1.0, 
    'palette': ['FFFFFF', 'DF923D', 'F1B555', 'FCD163', '74A901', '529400', '207401']
}

ndmi2Vis = {
    'min': 2, 
    'max': 10, 
    'palette': ["ffffff","85baf5","0b74ea","0d41e1","163eb6","1a3174"]
}

ndvi2Vis = {
    'min': 0, 
    'max': 10, 
    'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000', '529400', 
                '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301']
}

## Functions

In [42]:
def bufferGeo(resource, buffer):
    d1 = ee.Geometry.Point(resource.field.eeROI.centroid().coordinates())
    d2 = ee.Geometry.Point(ee.List(resource.field.eeROI.coordinates().get(0)).get(0))
    bufferVal = d1.distance(d2).add(buffer)
    bufferGeo = ee.Geometry.Point(resource.field.centroid).buffer(bufferVal)
    return bufferGeo

def renderNDMI(ndmi):
    ndmiRounded = ndmi.toFloat().multiply(10).toInt()
    ndmiFocal = ndmiRounded.toFloat().focal_median(kernelType="square", radius=5)
    ndmiRender = apgis.mask.generateRangeMask(image=ndmiFocal, low=2, high=10)
    return ndmiRender

def renderNDVI(ndvi):
    ndviRounded = ndvi.toFloat().multiply(10).toInt()
    ndviFocal = ndviRounded.toFloat().focal_median(kernelType="square", radius=5)
    ndviRender = apgis.mask.generateRangeMask(image=ndviFocal, low=0, high=10)
    return ndviRender

def renderVector(renderImage):
    render = renderImage.toInt().reduceToVectors(scale=1)
    return render

def layerCoding(renderImage):
    renderFeature = renderVector(renderImage)
    renderCoded = renderImage.reduceRegions(collection=renderFeature, reducer=ee.Reducer.mode().setOutputs(['layerID']), scale=1)
    return renderCoded

def mergeLayerPolygons(fCollection):
    layers = [int(float(i)) for i in fCollection.aggregate_histogram("layerID").getInfo().keys()]
    
    featureList = []
    for layer in layers:
        filterLayer = ee.Filter.rangeContains(field="layerID", minValue=layer, maxValue=layer)
        layerCollection = fCollection.filter(filterLayer)
        
        layerFeature = layerCollection.union(5).first().set({"layerID": layer})
        featureList.append(layerFeature)
        
    return ee.FeatureCollection(featureList)

def accumulateRawValue(indexImage, resource):
    latlon = ee.Image.pixelLonLat().reproject(indexImage.projection())
    coords = latlon.select(['longitude', 'latitude']).reduceRegion(reducer=ee.Reducer.toList(), geometry=resource.field.eeAOI, scale=10)
    
    lat = ee.List(coords.get('latitude'))
    lon = ee.List(coords.get('longitude'))
    coordinates = lon.zip(lat)
    
    def getFeature(latlon):
        point = ee.Geometry.Point(latlon)
        return ee.Feature(point)
    
    pointFeatures = ee.FeatureCollection(coordinates.map(getFeature))

    for index in indexImage.bandNames().getInfo():
        reduceIndexValues = ee.Reducer.first().setOutputs([index])
        pointFeatures = indexImage.reduceRegions(collection=pointFeatures, reducer=reduceIndexValues, scale=10)

    return pointFeatures

def setLatLon(feature):
    latlon = feature.geometry().coordinates()
    feature = feature.set({"longitude": latlon.get(0), "latitude": latlon.get(1)})
    return feature

def setArea(feature):
    area = feature.area(5)
    feature = feature.set({"area": area})
    return feature

def scoreBuilder(rawData):
    filterLowNDVI = ee.Filter.lte("NDVI", 0.4)
    lowCol = rawData.filter(filterLowNDVI)
    
    NDVIscore = lowCol.size().divide(rawData.size()).multiply(100).round()
    NDVImean = round(rawData.aggregate_mean("NDVI").getInfo(), 2)
    NDMImean = round(rawData.aggregate_mean("NDMI").getInfo(), 2)
    
    rawData = rawData.set({
        "NDVI Score": NDVIscore, 
        "NDVI Mean": NDVImean, 
        "NDMI Mean": NDMImean
    })
    
    return rawData

def getCleanSZ(ndviLayerCol, resource):
    filterSZ = ee.Filter.rangeContains(field="layerID", minValue=0, maxValue=3)
    szLayers = ndviLayerCol.filter(filterSZ)
    
    def setSZLayer(feature):
        feature = feature.set({"layerID": 1})
        return feature

    szLayered = szLayers.map(setSZLayer)
    
    szImage = szLayered.reduceToImage(['layerID'], ee.Reducer.first().setOutputs(['SZ']))
    cleanSZ = szImage.reduceToVectors(geometry=resource.field.eeAOI, scale=1) #something fishy here
    
    return cleanSZ
    
def areaLayers(bigSZLayers, threshArea):
    bigSZFeatures = bigSZLayers.toList(bigSZLayers.size())
    
    def makeFeatureCollectionList(feature, imageList):
        fCollection = ee.FeatureCollection([feature])
        image = fCollection.reduceToImage(['label'], ee.Reducer.first().setOutputs(['SZ'])).clip(feature)
        imageList = ee.List(imageList).add(image)
        return imageList

    clusterImageList = bigSZFeatures.iterate(makeFeatureCollectionList, ee.List([]))
    return clusterImageList

def fixSNIC(feature):
    featureOld = ee.Feature(feature)
    featureNew = ee.Feature(featureOld.geometry())

    mean = ee.Number(featureOld.get("mean"))
    layerID = mean.multiply(10).int()
    
    featureNew = featureNew.set({"layerID": layerID, "meanNDVI": mean})
    return featureNew

def splitSZLayers(areaSZ, threshArea):
    filterBigArea = ee.Filter.gte("area", threshArea)
    filterSmallArea = ee.Filter.lt("area", threshArea)

    bigSZ = areaSZ.filter(filterBigArea)
    smallSZ = areaSZ.filter(filterSmallArea)

    return bigSZ, smallSZ

def getClusterFeatureList(imageGeo, fList):
    fList = ee.List(fList)
    clusters = imageGeo.reduceToVectors(geometry=imageGeo.geometry(), scale=1)
    clusters = clusters.toList(clusters.size())
    fList = fList.add(clusters)
    return fList

def makeSZRender(ndviLayerCol, ndvi, resource):
    cleanSZ = getCleanSZ(ndviLayerCol, resource)
    areaSZ = cleanSZ.map(setArea)
    
    threshArea = resource.field.eeAOI.area(5).multiply(0.2).round().getInfo()
    threshPixel = round(threshArea/100)

    bigSZLayers, smallSZLayers = splitSZLayers(areaSZ, threshArea)
    
    clusterImageList = areaLayers(bigSZLayers, threshArea)
    clusterCollection = ee.ImageCollection.fromImages(clusterImageList)
    
    def applySNIC(clusterImage):
        seeds = ee.Algorithms.Image.Segmentation.seedGrid(size=round(threshPixel/4), gridType='hex')
        geoClusterImage = ee.Algorithms.Image.Segmentation.SNIC(image=clusterImage, size=round(threshPixel/2), compactness=5, seeds=seeds).select(['clusters']).clip(clusterImage.geometry())
        return geoClusterImage
    
    geoClusters = clusterCollection.map(applySNIC)    
    
    clusterFeatureList = ee.List(geoClusters.iterate(getClusterFeatureList, ee.List([]))).flatten()
    SNICCol = ee.FeatureCollection(clusterFeatureList).merge(smallSZLayers)
    SNICCol = ndvi.reduceRegions(collection=SNICCol, reducer=ee.Reducer.mean()).map(fixSNIC)
    
    return SNICCol

## Generate NDVI and NDMI

In [43]:
ndvi = apgis.index.generateIndex(image=image, sensor="L2A", index="NDVI").clip(resource.field.eeAOI)
ndmi = apgis.index.generateIndex(image=image, sensor="L2A", index="NDMI").clip(bufferGeo(resource, 50))

## Generate the Render base images

In [44]:
ndviRender = renderNDVI(ndvi)
ndmiRender = renderNDMI(ndmi)

## Generate Layer Coded Render Vectors

In [45]:
ndviLayerRender = layerCoding(ndviRender)
ndmiLayerRender = layerCoding(ndmiRender)

## Merge Layer Polygons

In [46]:
ndviLayerCol = mergeLayerPolygons(ndviLayerRender)
ndmiLayerCol = mergeLayerPolygons(ndmiLayerRender)

# print("NDVI", ndviLayerCol.aggregate_histogram("layerID").getInfo())
# print("NDMI", ndmiLayerCol.aggregate_histogram("layerID").getInfo())

NDVI {'1': 1, '2': 1, '3': 1, '4': 2, '5': 1, '6': 1, '7': 1}
NDMI {'2': 1, '3': 1, '4': 1, '5': 1, '6': 1}


## Combine Indexes and Accumulate Raw Values

In [47]:
indexes = ndvi.addBands(ndmi)
rawData = accumulateRawValue(indexes, resource)
scoredData = scoreBuilder(rawData)

## StressZone Render

In [48]:
SZRender = makeSZRender(ndviLayerCol, ndvi, resource)

## Export Build
## For reference only
```
exportsReference = {
    "Resource": resource.geojson.geojson,
    "Raw Data": rawData,
    "Render NDVI": ndviLayerCol,
    "Render NDMI": ndmiLayerCol,
    "Render SZ": SZRender
}
```
## Export

In [2]:
exportList = {
    "RawData": rawData, 
    "NDVIRender": ndviLayerCol, 
    "NDMIRender": ndmiLayerCol, 
    "SZRender": SZRender
}

NameError: name 'rawData' is not defined

In [1]:
tasklist = []
for exportName, exportObj in exportList.items():
    filePrefix = f"{remotefileName}-2-TestExportAcq"
    task = ee.batch.Export.table.toCloudStorage(collection= exportObj, 
                                                bucket="antpod-apgis-exports", 
                                                fileNamePrefix=f"{filePrefix}-{exportName}",
                                                fileFormat= 'GeoJSON')
    tasklist.append(task)

NameError: name 'exportList' is not defined

In [51]:
for task in tasklist:
    task.start()

In [92]:
tasklist[3].status()

{'state': 'COMPLETED',
 'description': 'myExportTableTask',
 'creation_timestamp_ms': 1599583962253,
 'update_timestamp_ms': 1599584034338,
 'start_timestamp_ms': 1599584026889,
 'task_type': 'EXPORT_FEATURES',
 'destination_uris': ['https://console.developers.google.com/storage/browser/antpod-apgis-exports/'],
 'id': 'OGHJC44XJ4NVL4JJRSLMQ6D4',
 'name': 'projects/earthengine-legacy/operations/OGHJC44XJ4NVL4JJRSLMQ6D4'}